In [1]:
%%capture
!pip install aequitas

In [5]:
from aequitas.group import Group

In [6]:
import numpy as np

import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegressionCV

# Tarea 1: Pipeline Aequitas

El objetivo de esta tarea es evaluar el uso del toolkit Aequitas para obtener métricas de grupo. En particular, trabajarán con los datos de riesgo crediticio alemán. A continuación se lista una descripción de los datos:

**Columnas Numéricas**

- `duration`: Duración en meses. Rango (4,72).
- `credit_amount`: Cantidad de crédito solicitada. Rango (250, 18424) en DM - Marco alemán.
- `installment_commitment`: Tasa de cuota en porcentaje del ingreso disponible. Rango (1,4).
- `residence_since`: Tiempo de residencia actual. Rango (1,4).
- `age`: Edad en años. Rango (19, 75).
- `existing_credits`: Número de créditos existentes en este banco. Rango (1,4) en DM - Marco alemán.
- `num_dependents`: Número de personas responsables de proveer el mantenimiento. Rango (1,2).

**Columnas Categóricas**

- `checking_status`: Valores `'0<=X<200', '<0', '>=200', 'no checking'`.
- `credit_history`: Historial crediticio del solicitante. Valores `['all paid', 'critical/other existing credit', 'delayed previously', 'existing paid', 'no credits/all paid']`.
- `purpose`: Motivo por el cual el solicitante solicitó un préstamo. Valores `['business', 'domestic appliance', 'education', 'furniture/equipment', 'new car', 'other', 'radio/tv', 'repairs', 'retraining', 'used car']`.
- `savings_status`: Cuenta de ahorros/bonos. Valores `['100<=X<500', '500<=X<1000', '<100', '>=1000', 'no known savings']`.
- `employment`: Empleo actual desde (en años). Valores `['1<=X<4', '4<=X<7', '<1', '>=7', 'unemployed']`.
- `other_parties`: Otros deudores / garantes. Valores `['co applicant', 'guarantor', 'none']`.
- `property_magnitude`: Bienes del solicitante. Valores `['car', 'life insurance', 'no known property', 'real estate']`.
- `other_payment_plans`: Otros planes de pago a plazos. Valores `['bank', 'none', 'stores']`.
- `housing`: Situación de vivienda del solicitante. Valores `['for free', 'own', 'rent']`.
- `job`: Categorías de empleo definidas por el banco. Valores `['high qualif/self emp/mgmt', 'skilled', 'unemp/unskilled non res', 'unskilled resident']`.
- `own_telephone`: Si hay un teléfono registrado a nombre del cliente. Valores `['none', 'yes']`.
- `foreign_worker`: Variable protegida. Valores `['no', 'yes']`.
- `sex`: Variable protegida. Valores `['female', 'male']`.
- `marital_status`: Estado personal. Valores `['div/dep/mar', 'div/sep', 'mar/wid', 'single']`.

**Etiqueta (variable objetivo)**

- `credit-risk`: `'good'` (favorable) o `'bad'` (desfavorable).

En este notebook se implementa, a partir de estos datos, un modelo simple de Regresión Logística el cual es utilizado para predecir el riesgo crediticio (bueno/malo), por lo tanto ustedes **NO DEBEN** implementar un modelo de clasificación, sino evaluar las métricas de grupo del modelo que se les entrega.

Esta tarea es **individual**. 

In [56]:
# En esta celda de código se desarrolla el modelo de Regresión Logística.

df = pd.read_csv('german_risk.csv')
df1 = df.copy()

# Binarización de algunas variables.

df['sex'] = df['sex'].map({'male': 1, 'female': 0})
df['age_cat'] = df['age_cat'].map({'aged': 1, 'young': 0})
df['foreign_worker'] = df['foreign_worker'].map({'no': 1, 'yes': 0})
df['credit-risk'] = df['credit-risk'].map({'good': 1, 'bad': 0})

# Separamos X e Y.

X = df.loc[:, df.columns != 'credit-risk']
y = df.loc[:, df.columns == 'credit-risk']

# Obtenemos variables dummies.

catcols = X.select_dtypes(exclude='number').columns
ignore = ['sex', 'age_cat', 'foreign_worker']
for catcol in catcols:
    if catcol in ignore:
        pass
    else:
        dummies = pd.get_dummies(X[catcol])
        X = pd.concat([X, dummies], axis=1).drop(columns=[catcol])


# Datos de entrenamiento y prueba

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=908)

# No utilizaremos la edad

X_train.drop(columns=['age'], inplace=True)
X_test.drop(columns=['age'], inplace=True)

# Regresión logística

lr = LogisticRegressionCV(solver='liblinear', cv=10, random_state=908)
lr.fit(X_train, np.ravel(y_train))

# Predicciones del modelo

y_pred_proba = lr.predict_proba(X_test)[:,1]
y_pred = y_pred_proba >= 0.5

# DataFrame Final

to_eval = X_test.copy()
to_eval['score'] = y_pred
to_eval['label'] = y_test

In [57]:
to_eval.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 200 entries, 298 to 219
Data columns (total 63 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   sex                             200 non-null    int64  
 1   age_cat                         200 non-null    int64  
 2   foreign_worker                  200 non-null    int64  
 3   duration                        200 non-null    float64
 4   credit_amount                   200 non-null    float64
 5   installment_commitment          200 non-null    float64
 6   residence_since                 200 non-null    float64
 7   existing_credits                200 non-null    float64
 8   num_dependents                  200 non-null    float64
 9   0<=X<200                        200 non-null    uint8  
 10  <0                              200 non-null    uint8  
 11  >=200                           200 non-null    uint8  
 12  no checking                     20

## 1. Prepara los datos `to_eval` para poder evaluarlos con Aequitas.

Para ello, define la función
```python
def pre_processing(to_eval):
    ...
    return df
```
que recibe los datos y los pre-procesa para utilizarlos en Aequitas. La función debe retornar el DataFrame obtenido. Puedes encontrar más detalles en la [documentación de Aequitas](https://dssg.github.io/aequitas/input_data.html#Input-data-for-Python-package). Utiliza los atributos `sex`, `age_cat` y `foreign_worker`. Asegúrese de mapear los valores de los distintos grupos a sus valores originales (i.e. `1` $\rightarrow$ `male`). Use los mismos valores que en los datos originales.

In [58]:
def pre_processing(to_eval):
    # your code here
    """
    Considerar transformaciones anteriores: 
    
    df['sex'] = df['sex'].map({'male': 1, 'female': 0})
    df['age_cat'] = df['age_cat'].map({'aged': 1, 'young': 0})
    df['foreign_worker'] = df['foreign_worker'].map({'no': 1, 'yes': 0})
    df['credit-risk'] = df['credit-risk'].map({'good': 1, 'bad': 0})    
    """
    # Llevamos los valores a los datos originales
    to_eval_original = to_eval.copy()
    to_eval_original['sex'] = to_eval_original['sex'].map({1: 'male', 0: 'female'})
    to_eval_original['age_cat'] = to_eval_original['age_cat'].map({1: 'aged', 0: 'young'})
    to_eval_original['foreign_worker'] = to_eval_original['foreign_worker'].map({1: 'no', 0: 'yes'})
    columns_to_use = ['sex', 'age_cat', 'foreign_worker', 'score', 'label']
    to_eval_original = to_eval_original[columns_to_use]

    # Transformamos para usar en Aequitas
    [columns_to_use]
    cat_cols = to_eval_original.select_dtypes(include='object').columns
    for col in cat_cols:
        to_eval_original[col] = to_eval_original[col].astype(str)

    to_eval_original['label_value'] = to_eval_original['label']
    to_eval_original.drop(columns='label', inplace=True)
    to_eval_original['score'] = to_eval_original['score'].map({True: 1, False: 0})

    return to_eval_original

In [59]:
assert pre_processing(to_eval)['sex'].dtype == 'object'
assert 'male' in pre_processing(to_eval)['sex'].unique()
assert pre_processing(to_eval)['score'].dtype in ['int', 'float']

## 2. Calcula las métricas grupales con Aequitas.

Para ello, utiliza la clase `Group()` de Aequitas. Define la función
```python
def group_metrics(to_eval):
    ...
    return df
```
que recibe el DataFrame ya pre-procesado en la pregunta anterior y retorna el DataFrame de las métricas grupales (fpr, fnr, fdr, for, etc).

In [62]:
def group_metrics(to_eval):
    # your code here
    bias = Group()
    xtab, _ = bias.get_crosstabs(to_eval)
    return xtab
    

In [63]:
# Tests
female = group_metrics(pre_processing(to_eval)).query("attribute_value == 'female'").reset_index(drop=True)
male = group_metrics(pre_processing(to_eval)).query("attribute_value == 'male'").reset_index(drop=True)

assert group_metrics(pre_processing(to_eval)).shape == (6, 27)